<a href="https://colab.research.google.com/github/onism/ML-learning/blob/master/SpeechRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torchaudio

     |████████████████████████████████| 3.2MB 9.1MB/s 


Two of the most popular end-to-end models are Deep Speech by Baidu, and Listen Attend Spell (LAS) by Google.
Deep Speech uses the Connectionist Temporal Classification (CTC) loss function to predict the speech transcript. LAS uses a sequence to sequence network architecture for its predictions.

In [4]:
import torchaudio 
train_dataset = torchaudio.datasets.LIBRISPEECH('./', url='train-clean-100', download=True)
test_dataset = torchaudio.datasets.LIBRISPEECH('./', url='test-clean', download=True)

Spectrogram Augmentation(SpecAugment), to be a much simpler and more effective approach. 
In PyTorch, we can use the torchaudio function FrequencyMasking to mask out the frequency dimension, and TimeMasking for the time dimension

In [0]:
class TextTransorm:
    'Maps char to integers and vice versa'
    def __init__(self):
        char_map_str = """
        ' 0
        <SPACE> 1
        a 2
        b 3
        c 4
        d 5
        e 6
        f 7
        g 8
        h 9
        i 10
        j 11
        k 12
        l 13
        m 14
        n 15
        o 16
        p 17
        q 18
        r 19
        s 20
        t 21
        u 22
        v 23
        w 24
        x 25
        y 26
        z 27
        """
        self.char_map = {}
        self.index_map = {}
        for line in char_map_str.strip().split('\n'):
            ch, index = line.split()
            self.char_map[ch] = int(index)
            self.index_map[int(index)] = ch 
        self.index_map[1] = ' '
    
    def text_to_int(self, text):
        int_seq = []
        for c in text:
            if c == ' ':
                ch = self.char_map['']
            else:
                ch = self.char_map[c]
            int_seq.append(ch)
        return int_seq 
    
    def int_to_text(self, labels):
        string = []
        for i in labels:
            string.append(self.index_map[i])
        return ''.joint(string).replace('', ' ')


In [0]:
from torch import nn
train_audio_transforms = nn.Sequential(
    torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128),
    torchaudio.transforms.FrequencyMasking(freq_mask_param=15),
    torchaudio.transforms.TimeMasking(time_mask_param=35)
)
valid_audio_transforms = torchaudio.transforms.MelSpectrogram()

text_transform  = TextTransorm()

In [0]:
import torch 
def data_preprocessing(data, data_type='train'):
    spectrograms = []
    labels = []
    input_len = []
    label_len = []
    for (waveform, _, utterance, _, _, _) in data:
        if data_type == 'train':
            spec = train_audio_transforms(waveform).squeeze(0).transpose(0,1)
        else:
            spec = valid_audio_transforms(waveform).squeeze(0).transpose(0,1)
        spectrograms.append(spec)
        label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
        labels.append(label)
        input_len.append(spec.shape[0]//2)
        label_len.append(len(label))
    spectrograms = nn.utils.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2,3)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)
    return spectrograms, labels, input_len, label_len

![ Deep Speech 2 architecture](https://miro.medium.com/max/1400/0*1TokC95mn1dVllEx.png)

In [0]:
# https://towardsdatascience.com/customer-case-study-building-an-end-to-end-speech-recognition-model-in-pytorch-with-assemblyai-473030e47c7c

